In [1]:
from textgenrnn import textgenrnn
from datetime import datetime
import os

Using TensorFlow backend.


In [2]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 50,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [3]:
file_name = "wa2.txt"
model_name = 'WA'   # change to set file name of resulting trained models/texts

In [4]:
# Code for importing model from files
textgen = textgenrnn(weights_path='WA_weights.hdf5',
                       vocab_path='WA_vocab.json',
                       config_path='WA_config.json')

In [4]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

44,792 texts collected.
Training new model w/ 4-layer, 128-cell Bidirectional LSTMs
Training on 1,412,001 character sequences.
Epoch 1/20


 314/1378 [=====>........................] - ETA: 1:53:55 - loss: 4.52 - ETA: 1:02:16 - loss: 4.68 - ETA: 44:59 - loss: 7.2178 - ETA: 36:22 - loss: 7.73 - ETA: 31:12 - loss: 7.44 - ETA: 27:44 - loss: 7.79 - ETA: 25:18 - loss: 8.25 - ETA: 23:26 - loss: 8.57 - ETA: 22:00 - loss: 8.83 - ETA: 20:50 - loss: 9.12 - ETA: 19:54 - loss: 9.24 - ETA: 19:06 - loss: 8.89 - ETA: 18:25 - loss: 8.51 - ETA: 17:51 - loss: 8.24 - ETA: 17:21 - loss: 7.95 - ETA: 16:55 - loss: 7.69 - ETA: 16:31 - loss: 7.48 - ETA: 16:11 - loss: 7.28 - ETA: 15:53 - loss: 7.07 - ETA: 15:36 - loss: 6.89 - ETA: 15:21 - loss: 6.72 - ETA: 15:06 - loss: 6.56 - ETA: 14:54 - loss: 6.42 - ETA: 14:42 - loss: 6.29 - ETA: 14:31 - loss: 6.17 - ETA: 14:21 - loss: 6.06 - ETA: 14:12 - loss: 5.95 - ETA: 14:03 - loss: 5.85 - ETA: 13:55 - loss: 5.76 - ETA: 13:48 - loss: 5.68 - ETA: 13:41 - loss: 5.60 - ETA: 13:34 - loss: 5.53 - ETA: 13:27 - loss: 5.46 - ETA: 13:21 - loss: 5.39 - ETA: 13:16 - loss: 5.33 - ETA: 13:10 - loss: 5.27 - ETA: 13:05 - 

 942/1378 [===================>..........] - ETA: 5:54 - loss: 3.138 - ETA: 5:54 - loss: 3.137 - ETA: 5:53 - loss: 3.136 - ETA: 5:53 - loss: 3.135 - ETA: 5:52 - loss: 3.134 - ETA: 5:52 - loss: 3.133 - ETA: 5:51 - loss: 3.132 - ETA: 5:51 - loss: 3.131 - ETA: 5:50 - loss: 3.130 - ETA: 5:50 - loss: 3.130 - ETA: 5:49 - loss: 3.129 - ETA: 5:49 - loss: 3.128 - ETA: 5:48 - loss: 3.128 - ETA: 5:48 - loss: 3.127 - ETA: 5:47 - loss: 3.126 - ETA: 5:47 - loss: 3.126 - ETA: 5:46 - loss: 3.125 - ETA: 5:46 - loss: 3.124 - ETA: 5:45 - loss: 3.123 - ETA: 5:45 - loss: 3.122 - ETA: 5:45 - loss: 3.121 - ETA: 5:44 - loss: 3.121 - ETA: 5:44 - loss: 3.120 - ETA: 5:43 - loss: 3.119 - ETA: 5:43 - loss: 3.118 - ETA: 5:42 - loss: 3.117 - ETA: 5:42 - loss: 3.116 - ETA: 5:41 - loss: 3.116 - ETA: 5:41 - loss: 3.115 - ETA: 5:40 - loss: 3.114 - ETA: 5:40 - loss: 3.113 - ETA: 5:39 - loss: 3.112 - ETA: 5:39 - loss: 3.111 - ETA: 5:38 - loss: 3.110 - ETA: 5:38 - loss: 3.109 - ETA: 5:37 - loss: 3.108 - ETA: 5:37 - loss: 3

1256/1378 [==========================>...] - ETA: 3:25 - loss: 2.846 - ETA: 3:24 - loss: 2.845 - ETA: 3:24 - loss: 2.844 - ETA: 3:23 - loss: 2.843 - ETA: 3:23 - loss: 2.842 - ETA: 3:22 - loss: 2.841 - ETA: 3:22 - loss: 2.840 - ETA: 3:21 - loss: 2.838 - ETA: 3:21 - loss: 2.837 - ETA: 3:20 - loss: 2.836 - ETA: 3:20 - loss: 2.835 - ETA: 3:19 - loss: 2.834 - ETA: 3:19 - loss: 2.833 - ETA: 3:18 - loss: 2.832 - ETA: 3:18 - loss: 2.831 - ETA: 3:17 - loss: 2.830 - ETA: 3:17 - loss: 2.829 - ETA: 3:17 - loss: 2.828 - ETA: 3:16 - loss: 2.827 - ETA: 3:16 - loss: 2.826 - ETA: 3:15 - loss: 2.825 - ETA: 3:15 - loss: 2.823 - ETA: 3:14 - loss: 2.822 - ETA: 3:14 - loss: 2.821 - ETA: 3:13 - loss: 2.820 - ETA: 3:13 - loss: 2.819 - ETA: 3:12 - loss: 2.818 - ETA: 3:12 - loss: 2.817 - ETA: 3:11 - loss: 2.816 - ETA: 3:11 - loss: 2.815 - ETA: 3:10 - loss: 2.814 - ETA: 3:10 - loss: 2.813 - ETA: 3:09 - loss: 2.812 - ETA: 3:09 - loss: 2.811 - ETA: 3:08 - loss: 2.810 - ETA: 3:08 - loss: 2.809 - ETA: 3:08 - loss: 2

1378/1378 [==============================] - ETA: 56s - loss: 2.56 - ETA: 56s - loss: 2.56 - ETA: 55s - loss: 2.56 - ETA: 55s - loss: 2.56 - ETA: 55s - loss: 2.56 - ETA: 54s - loss: 2.55 - ETA: 54s - loss: 2.55 - ETA: 53s - loss: 2.55 - ETA: 53s - loss: 2.55 - ETA: 52s - loss: 2.55 - ETA: 52s - loss: 2.55 - ETA: 51s - loss: 2.55 - ETA: 51s - loss: 2.55 - ETA: 50s - loss: 2.55 - ETA: 50s - loss: 2.55 - ETA: 49s - loss: 2.55 - ETA: 49s - loss: 2.55 - ETA: 48s - loss: 2.55 - ETA: 48s - loss: 2.54 - ETA: 47s - loss: 2.54 - ETA: 47s - loss: 2.54 - ETA: 47s - loss: 2.54 - ETA: 46s - loss: 2.54 - ETA: 46s - loss: 2.54 - ETA: 45s - loss: 2.54 - ETA: 45s - loss: 2.54 - ETA: 44s - loss: 2.54 - ETA: 44s - loss: 2.54 - ETA: 43s - loss: 2.54 - ETA: 43s - loss: 2.54 - ETA: 42s - loss: 2.54 - ETA: 42s - loss: 2.53 - ETA: 41s - loss: 2.53 - ETA: 41s - loss: 2.53 - ETA: 40s - loss: 2.53 - ETA: 40s - loss: 2.53 - ETA: 39s - loss: 2.53 - ETA: 39s - loss: 2.53 - ETA: 39s - loss: 2.53 - ETA: 38s - loss: 2.

...

1378/1378 [==============================] - ETA: 56s - loss: 0.67 - ETA: 56s - loss: 0.67 - ETA: 55s - loss: 0.67 - ETA: 55s - loss: 0.67 - ETA: 54s - loss: 0.67 - ETA: 54s - loss: 0.67 - ETA: 53s - loss: 0.67 - ETA: 53s - loss: 0.67 - ETA: 52s - loss: 0.67 - ETA: 52s - loss: 0.67 - ETA: 51s - loss: 0.67 - ETA: 51s - loss: 0.67 - ETA: 50s - loss: 0.67 - ETA: 50s - loss: 0.67 - ETA: 49s - loss: 0.67 - ETA: 49s - loss: 0.67 - ETA: 49s - loss: 0.67 - ETA: 48s - loss: 0.67 - ETA: 48s - loss: 0.67 - ETA: 47s - loss: 0.67 - ETA: 47s - loss: 0.67 - ETA: 46s - loss: 0.67 - ETA: 46s - loss: 0.67 - ETA: 45s - loss: 0.67 - ETA: 45s - loss: 0.67 - ETA: 44s - loss: 0.67 - ETA: 44s - loss: 0.67 - ETA: 43s - loss: 0.67 - ETA: 43s - loss: 0.67 - ETA: 42s - loss: 0.67 - ETA: 42s - loss: 0.67 - ETA: 42s - loss: 0.67 - ETA: 41s - loss: 0.67 - ETA: 41s - loss: 0.67 - ETA: 40s - loss: 0.67 - ETA: 40s - loss: 0.67 - ETA: 39s - loss: 0.67 - ETA: 39s - loss: 0.67 - ETA: 38s - loss: 0.67 - ETA: 38s - loss: 0.

 314/1378 [=====>........................] - ETA: 10:52 - loss: 0.65 - ETA: 10:48 - loss: 0.61 - ETA: 10:46 - loss: 0.64 - ETA: 10:45 - loss: 0.65 - ETA: 10:43 - loss: 0.65 - ETA: 10:43 - loss: 0.66 - ETA: 10:40 - loss: 0.65 - ETA: 10:40 - loss: 0.65 - ETA: 10:40 - loss: 0.65 - ETA: 10:39 - loss: 0.65 - ETA: 10:39 - loss: 0.65 - ETA: 10:38 - loss: 0.65 - ETA: 10:37 - loss: 0.65 - ETA: 10:37 - loss: 0.65 - ETA: 10:36 - loss: 0.65 - ETA: 10:36 - loss: 0.65 - ETA: 10:35 - loss: 0.66 - ETA: 10:35 - loss: 0.66 - ETA: 10:35 - loss: 0.65 - ETA: 10:34 - loss: 0.65 - ETA: 10:33 - loss: 0.65 - ETA: 10:33 - loss: 0.65 - ETA: 10:32 - loss: 0.65 - ETA: 10:32 - loss: 0.65 - ETA: 10:32 - loss: 0.65 - ETA: 10:31 - loss: 0.65 - ETA: 10:30 - loss: 0.65 - ETA: 10:30 - loss: 0.65 - ETA: 10:29 - loss: 0.66 - ETA: 10:29 - loss: 0.65 - ETA: 10:28 - loss: 0.65 - ETA: 10:28 - loss: 0.65 - ETA: 10:28 - loss: 0.66 - ETA: 10:27 - loss: 0.66 - ETA: 10:26 - loss: 0.66 - ETA: 10:26 - loss: 0.66 - ETA: 10:26 - loss: 

 942/1378 [===================>..........] - ETA: 5:50 - loss: 0.652 - ETA: 5:49 - loss: 0.652 - ETA: 5:49 - loss: 0.651 - ETA: 5:48 - loss: 0.651 - ETA: 5:48 - loss: 0.652 - ETA: 5:47 - loss: 0.652 - ETA: 5:47 - loss: 0.652 - ETA: 5:46 - loss: 0.652 - ETA: 5:46 - loss: 0.652 - ETA: 5:45 - loss: 0.652 - ETA: 5:45 - loss: 0.652 - ETA: 5:44 - loss: 0.652 - ETA: 5:44 - loss: 0.652 - ETA: 5:43 - loss: 0.652 - ETA: 5:43 - loss: 0.652 - ETA: 5:43 - loss: 0.652 - ETA: 5:42 - loss: 0.652 - ETA: 5:42 - loss: 0.652 - ETA: 5:41 - loss: 0.652 - ETA: 5:41 - loss: 0.652 - ETA: 5:40 - loss: 0.652 - ETA: 5:40 - loss: 0.651 - ETA: 5:39 - loss: 0.651 - ETA: 5:39 - loss: 0.651 - ETA: 5:38 - loss: 0.651 - ETA: 5:38 - loss: 0.651 - ETA: 5:37 - loss: 0.651 - ETA: 5:37 - loss: 0.651 - ETA: 5:36 - loss: 0.652 - ETA: 5:36 - loss: 0.651 - ETA: 5:35 - loss: 0.651 - ETA: 5:35 - loss: 0.651 - ETA: 5:35 - loss: 0.651 - ETA: 5:34 - loss: 0.651 - ETA: 5:34 - loss: 0.651 - ETA: 5:33 - loss: 0.651 - ETA: 5:33 - loss: 0

1256/1378 [==========================>...] - ETA: 3:23 - loss: 0.652 - ETA: 3:22 - loss: 0.652 - ETA: 3:22 - loss: 0.652 - ETA: 3:21 - loss: 0.652 - ETA: 3:21 - loss: 0.652 - ETA: 3:20 - loss: 0.652 - ETA: 3:20 - loss: 0.652 - ETA: 3:19 - loss: 0.652 - ETA: 3:19 - loss: 0.652 - ETA: 3:19 - loss: 0.652 - ETA: 3:18 - loss: 0.652 - ETA: 3:18 - loss: 0.652 - ETA: 3:17 - loss: 0.652 - ETA: 3:17 - loss: 0.652 - ETA: 3:16 - loss: 0.652 - ETA: 3:16 - loss: 0.652 - ETA: 3:15 - loss: 0.652 - ETA: 3:15 - loss: 0.652 - ETA: 3:14 - loss: 0.652 - ETA: 3:14 - loss: 0.652 - ETA: 3:13 - loss: 0.652 - ETA: 3:13 - loss: 0.652 - ETA: 3:12 - loss: 0.652 - ETA: 3:12 - loss: 0.652 - ETA: 3:12 - loss: 0.652 - ETA: 3:11 - loss: 0.652 - ETA: 3:11 - loss: 0.652 - ETA: 3:10 - loss: 0.652 - ETA: 3:10 - loss: 0.652 - ETA: 3:09 - loss: 0.652 - ETA: 3:09 - loss: 0.652 - ETA: 3:08 - loss: 0.652 - ETA: 3:08 - loss: 0.652 - ETA: 3:07 - loss: 0.652 - ETA: 3:07 - loss: 0.652 - ETA: 3:06 - loss: 0.652 - ETA: 3:06 - loss: 0

1378/1378 [==============================] - ETA: 56s - loss: 0.65 - ETA: 56s - loss: 0.65 - ETA: 55s - loss: 0.65 - ETA: 55s - loss: 0.65 - ETA: 54s - loss: 0.65 - ETA: 54s - loss: 0.65 - ETA: 53s - loss: 0.65 - ETA: 53s - loss: 0.65 - ETA: 52s - loss: 0.65 - ETA: 52s - loss: 0.65 - ETA: 51s - loss: 0.65 - ETA: 51s - loss: 0.65 - ETA: 50s - loss: 0.65 - ETA: 50s - loss: 0.65 - ETA: 49s - loss: 0.65 - ETA: 49s - loss: 0.65 - ETA: 49s - loss: 0.65 - ETA: 48s - loss: 0.65 - ETA: 48s - loss: 0.65 - ETA: 47s - loss: 0.65 - ETA: 47s - loss: 0.65 - ETA: 46s - loss: 0.65 - ETA: 46s - loss: 0.65 - ETA: 45s - loss: 0.65 - ETA: 45s - loss: 0.65 - ETA: 44s - loss: 0.65 - ETA: 44s - loss: 0.65 - ETA: 43s - loss: 0.65 - ETA: 43s - loss: 0.65 - ETA: 42s - loss: 0.65 - ETA: 42s - loss: 0.65 - ETA: 42s - loss: 0.65 - ETA: 41s - loss: 0.65 - ETA: 41s - loss: 0.65 - ETA: 40s - loss: 0.65 - ETA: 40s - loss: 0.65 - ETA: 39s - loss: 0.65 - ETA: 39s - loss: 0.65 - ETA: 38s - loss: 0.65 - ETA: 38s - loss: 0.

In [5]:
textgen.generate_samples(temperatures=[0.2, 0.5, 1.0, 1.2, 1.5])

####################
Temperature: 0.2
####################
Kevin Algera: Dat is t nog wel een keer dat ik het m niet wat laten want dan komt ook nog wel een goeie tijd hebben

Kevin Algera: Heb je nog een beetje gezegd

Kevin Algera: Dan moet je wel in t vliegtuig wordt t niet

####################
Temperature: 0.5
####################
Kevin Algera: Ja klopt

Kevin Algera: Gewoon andere dagen

Kevin Algera: Haha

####################
Temperature: 1.0
####################
Yoram Carboex: Jep

Kevin Algera: Hz gaat dat dag

Gijs Cunnen: Kan ook mee?

####################
Temperature: 1.2
####################
Kevin Algera: <Media omitted>

Kevin Algera: Casa

Kevin Algera: Das a ahademicsed uit

####################
Temperature: 1.5
####################
Kevin Algera: Hmmm aangevuld

Kevin Algera: Mijn bamre 

Thijs van der Putten: kun je morgen pas collen.



In [5]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix_list = [
    "Koen Niemeijer",
    "Kevin Algera",
    "Willem Smits",
    "Gijs Cunnen",
    "Thijs van der Putten",
    "Wescel Manders",
    "Thomas Bardoel",
    "Michiel Arts",
    "Yoram Carboex"
]

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000

# Generate for everyone
gen_file = '{}_gentext_{}.txt'.format(model_name, "")
textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix="",
                         n=3000,
                         max_gen_length=max_gen_length)

# Generate for rest
for prefix in prefix_list:
    gen_file = '{}_gentext_{}.txt'.format(model_name, prefix)
    textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)



100%|██████████████████████████████████████| 1000/1000 [07:16<00:00,  2.29it/s]
